In [41]:
#import everything that's necessary
import os
from lxml import etree

#get path where provenance XML file is stored
xml_path = input('Please enter the path to the provenance XML file on your computer.')
doc = etree.parse(xml_path)
record_elem = doc.findall('recordList/record')

Please enter the path to the provenance XML file on your computer.../Data/xml_data/provenance.xml


In [65]:
for elem in record_elem[:2]:
    #get the identifier
    identifier = elem.find('PersistentIdentifier')
    identifier = identifier.text
    #get the provenance text
    provenance = elem.find('provenance/provenance.project.txt')
    if provenance == None:
        continue
    provenance = provenance.text
    #split the provenance text into its separate chunks
    provenance_list = provenance.split(';')
    for elem in provenance_list:
        #see what kind of element it is and go to that function
        elem_old = elem
        if elem == '...':
            elem_new_format = '...'
            elem_new_readable = 'Previous provenance unknown.'
        elif 'Sale' in elem or 'sale' in elem:
            auction_item(elem)
            print('NOT YET IMPLEMENTED')
        elif 'Estate inventory' in elem or 'estate inventory' in elem:
            elem_new_format, elem_new_readable = estate_inv_item(elem)
        elif elem.startswith(' dealer') or elem.startswith('dealer') or elem.startswith('Dealer') or elem.startswith(' Dealer') or 'art market' in elem:
            art_market_dealer_item(elem)
        elif ('his widow' in elem and not 'estate' in elem) or ('her widow' in elem and not 'estate' in elem) or ('his son' in elem and not 'estate' in elem) or ('her son' in elem and not 'estate' in elem) or ('his daughter' in elem and not 'estate' in elem) or ('her daughter' in elem and not 'estate' in elem) or ('his niece' in elem and not 'estate' in elem) or ('her niece' in elem and not 'estate' in elem) or ('his nephew' in elem and not 'estate' in elem) or ('her nephew' in elem and not 'estate' in elem) or (' grand' in elem and not 'estate' in elem):
            #elem_new_format, elem_new_readable = inheritance_other_item(elem)
            print('NOT WORKING FULLY YET')
        elif 'by descent' in elem and not 'estate' in elem:
            elem_new_format, elem_new_readable = inheritance_descent_item(elem)
        elif elem.startswith('{') and elem.endswith('}'):
            #element is just a source
            elem_new_format = elem
            elem_new_readable = ''
        else:
            elem_new_format = 'Code for this type of item does not yet exist.'
            elem_new_readable = 'Code for this type of item does not yet exist.'
            
        print(elem_new_format)
        print(elem_new_readable)

Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
NOT YET IMPLEMENTED
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
Code for this type of item does not yet exist.
{RMA Archive, R.W. van Hoey Smith, lette

In [44]:
#function for all auction items
def auction_item(item):
    """
    Parse provenance items sold at an auction and return the new proper format, the readable sentence, and the format if the item is suspicious
    """
    #still need to combine these items to make sure they all work properly together
    pass

In [52]:
#function for all estate inventories
def estate_inv_item(item):
    """
    Parse provenance items found during an estate inventory and return the new proper format, the readable sentence, and the format if the item is suspicious
    """
    
    #possible parts of the provenance sentence
    name = ''
    name2 = '' # if there's a second owner
    BDdate = '' #birth-death date
    BDdate2 = ''
    extra = ''
    extra2 = ''
    place = ''
    place_extra = ''
    date = '' #date of inventory
    location = '' #location of work in house
    no = '' #no in inventory
    fol = ''
    p = ''
    attribution = ''
    description = ''
    source = ''
    start_source = ''
    #fix possible missing comma at the start
    test_item = item
    if 'estate inventory ' in item:
        item = item.replace('estate inventory', 'estate inventory,').replace('estate inventory of','estate inventory,')
    #saving the original, but fixed, item in case I need it
    start_item = item
    #print('Item:', start_item) 
    #certain or uncertain provenance
    certain_uncertain = item.split('(')
    if '?' in certain_uncertain[0]:
        #provenance data is uncertain
        proper_format = '? '
        readable_sent = 'Possibly part of '
    else:
        #provenance data is certain
        proper_format = ''
        readable_sent = 'Part of '
    
    #source
    #see if item starts with source but a { is missing
    check_split = item.split('estate')
    if '}' in check_split[0] and not '{' in check_split[0]:
        item = '{'+item
    #if item starts with source
    if item.startswith('{') or item.startswith(' {'):
        curly_split = item.split('}')
        start_source = curly_split[0].replace('{', '').strip()
    #if item ends with source
    if '{' in item and start_source == '':
        curly_split = item.split('{')
        source = curly_split[1].replace('}', '').strip()
    if start_source != '':
        sourceless_item = start_item.replace(start_source,'').strip()
    else:
        sourceless_item = start_item.replace(source,'').strip()
    if start_source != '':
        start_source = '{'+start_source+'}'
    if source != '':
        source = '{'+source+'}'
    
    #name owner
    comma_split = sourceless_item.split(',')
    #no owner name, owner name in previous provenance sentence
    if 'his ' in comma_split[0]:
        proper_format = f"{proper_format}his "
        readable_sent = f"{readable_sent}his "
    elif 'her ' in comma_split[0]:
        proper_format = f"{proper_format}her "
        readable_sent = f"{readable_sent}her "
    elif 'their ' in comma_split[0]:
        proper_format = f"{proper_format}their "
        readable_sent = f"{readable_sent}their "
    #owner name is mentioned
    else:
        name_elem = comma_split[1]
        if ' and ' in name_elem:
            two_names = name_elem.split('and')
            name_one = two_names[0]
            name = name_one.strip()
            if '(' in name_one:
                name_one = name_one.split('(')
                name = name_one[0].strip()
                #BDdate person 1
                BDdate = name_one[1].replace(')','').strip()
            name_two = two_names[1]
            name2 = name_two.strip()
            if '(' in name_two:
                name_two = name_two.split('(')
                name2 = name_two[0].strip()
                #BDdate person 2
                BDdate2 = name_two[1].replace(')','').strip()
        else:
            name = name_elem.strip()
            name_elem = name_elem.split('(')
            name = name_elem[0].strip()
            if 'House' in name or 'house' in name or 'Huis' in name or 'huis' in name or 'Castle' in name or 'castle' in name or 'Kasteel' in name or 'kasteel' in name or 'Paleis' in name or 'paleis' in name or 'palace' in name or 'Palace' in name:
                place = name
                name = ''
            if len(name_elem) > 1 and ('1' in name_elem[1] or '2' in name_elem[1]) and name !='':
                BDdate = name_elem[1].replace(')','').strip()
            if ('1' in comma_split[2] or '2' in comma_split[2]) and BDdate == '' and name !='':
                extra = comma_split[2]
                extra_elem = extra.split('(')
                extra = extra_elem[0].strip()
                BDdate = extra_elem[1].replace(')','').strip()
            if '1' in name_elem or '2' in name_elem and BDdate == '' and name !='':
                BDdate = name_elem[1].replace(')','').strip()
                
        #extra info owner
        if BDdate not in comma_split[1]:
            extra = comma_split[2]
            extra = extra.split('(')
            extra = extra[0].strip()
        #if extra info is about being a widow
        if ' widow ' in comma_split[2]:
            extra = comma_split[2].strip()
                
    #inventory date
    for item in comma_split:
        #no in inventory
        if 'no.' in item:
            no = item
            if '(' in item:
                split_item = item.split('(')
                no = split_item[0].strip()
        if 'fol.' in item:
            fol = item.strip()
        if 'p.' in item:
            p = item.strip()
        #actual inventory date
        item = item.split('(')
        if ('1' in item[0] or '2' in item[0]) and date == '' and not "'" in item[0] and not "‘" in item[0]:
            date = item[0]
            
    #description
    para_split = sourceless_item.split('(')
    for item in para_split:
        if "')" in item or "’)" in item:
            item = item.split(')')
            description = item[0].strip()  
        
            
    #attribution
    for item in comma_split:
        if 'without attribution' in item:
            attribution = 'without attribution'
        if ' as ' in item:
            attribution = item
            attribution = attribution.split('(')
            attribution = attribution[0].strip()
        if "'" in item or "‘" in item:
            item = item.split('(')
            if "'" in item[0] or "‘" in item[0]:
                attribution = item[0].strip()
            
    #location in the house
    for item in comma_split:
        if 'in the ' in item:
            item = item.split("(")
            location = item[0].strip()
        if 'Op de ' in item or 'op de ' in item:
            item = item.split('(')
            location = item[0].strip()
        if 'chamber' in item:
            item = item.split("(")
            location = item[0].strip()
            if "'" in location or "‘" in location:
                location = ''
    
    #place
    #index number of date
    for item in comma_split:
        if date in item and not "'" in item and not "‘" in item:
            date_index = comma_split.index(item)
            place = comma_split[(date_index - 1)]
    if 'near ' in place:
        place_extra = place
        place = comma_split[(date_index - 2)].strip()
    if 'estate inventory' in place:
        place = ''
    if place != '' and place_extra == '':
        place_index = comma_split.index(place)
        if BDdate not in comma_split[(place_index-1)] and BDdate2 not in comma_split[(place_index-1)] and name not in comma_split[(place_index-1)] and name2 not in comma_split[(place_index-1)]:
            place_extra = place
            place = comma_split[(place_index-1)].strip() 
        if 'near ' in comma_split[(place_index+1)]:
            place_extra = comma_split[(place_index+1)]
    #extra info owner 2
    if place != '':
        place_index = ' ' + place
        if '  ' in place_index:
            place_index = place_index.replace('  ',' ')
        place_index = comma_split.index(place_index)
        index = (place_index - 1)
        if name2 not in comma_split[index] and extra not in comma_split[index] and name not in comma_split[index]:
            extra2 = comma_split[(place_index -1)].strip()
    
    if name == 'William III':
        extra = 'Prince of Orange, King of England'
    if name == 'James II':
        extra = 'King of England'
    if date in place:
        place = ''
    
    #check just in case if everything is properly stripped
    start_source = start_source.strip()
    name = name.strip()
    BDdate = BDdate.strip()
    extra = extra.strip()
    name2 = name2.strip()
    BDdate2 = BDdate2.strip()
    extra2 = extra2.strip()
    place = place.strip()
    place_extra = place_extra.strip()
    date = date.strip()
    location = location.strip()
    fol = fol.strip()
    p = p.strip()
    no = no.strip()
    attribution = attribution.strip()
    description = description.strip()
    source = source.strip()
    
    #proper format and readable sentence
    proper_format = f"{proper_format}estate inventory"
    readable_sent = f"{readable_sent}estate inventory"
    #proper format
    if start_source != '':
        proper_format = f"{start_source} {proper_format}"
    if name != '':
        proper_format = f"{proper_format}, {name}"
    if BDdate != '':
        proper_format = f"{proper_format} ({BDdate})"
    if extra != '':
        proper_format = f"{proper_format}, {extra}"
    if name2 != '' and extra != '':
        proper_format = f"{proper_format}, and {name2}"
    if name2 != '' and extra == '':
        proper_format = f"{proper_format} and {name2}"
    if BDdate2 != '':
        proper_format = f"{proper_format} ({BDdate2})"
    if extra2 != '':
        proper_format = f"{proper_format}, {extra2}"
    if place != '':
        proper_format = f"{proper_format}, {place}"
    if place_extra != '':
        proper_format = f"{proper_format}, {place_extra}"
    if date != '':
        proper_format = f"{proper_format}, {date}"
    if location != '':
        proper_format = f"{proper_format}, {location}"
    if fol != '':
        proper_format = f"{proper_format}, {fol}"
    if p != '':
        proper_format = f"{proper_format}, {p}"
    if no != '':
        proper_format = f"{proper_format}, {no}"
    if attribution != '':
        proper_format = f"{proper_format}, {attribution}"
    if description != '':
        proper_format = f"{proper_format} ({description})"
    if source != '':
        proper_format = f"{proper_format} {source}"
    #print('Proper format:', proper_format)
    
    #readable sentence
    if name != '':
        readable_sent = f"{readable_sent} of {name}"
    if BDdate != '':
        readable_sent = f"{readable_sent} ({BDdate})"
    if extra != '':
        readable_sent = f"{readable_sent}, {extra}"
    if name2 != '' and extra != '':
        readable_sent = f"{readable_sent}, and {name2}"
    if name2 != '' and extra == '':
        readable_sent = f"{readable_sent} and {name2}"
    if BDdate2 != '':
        readable_sent = f"{readable_sent} ({BDdate2})"
    if extra2 != '':
        readable_sent = f"{readable_sent}, {extra2}"
    if place != '':
        readable_sent = f"{readable_sent}, {place}"
    if place_extra != '':
        readable_sent = f"{readable_sent}, {place_extra}"
    if date != '':
        readable_sent = f"{readable_sent}, {date}"
    if location != '':
        readable_sent = f"{readable_sent}, hanging {location}"
    if fol != '':
        readable_sent = f"{readable_sent}, {fol}"
    if p != '':
        readable_sent = f"{readable_sent}, {p}"
    if no != '':
        readable_sent = f"{readable_sent}, {no}"
    if attribution != '':
        readable_sent = f"{readable_sent}, {attribution}"
    if description != '':
        readable_sent = f"{readable_sent} ({description})"
    readable_sent = f"{readable_sent}."
    
    return proper_format, readable_sent
    

In [54]:
#function for all art market or dealer items
def art_market_dealer_item(item):
    """
    Parse provenance items sold at art market/dealer and return the new proper format, the readable sentence, and the format if the item is suspicious
    """
    
    test_item = item
    #art market dealer unknown
    if 'art market' in item:
        art_market = ''
        year = ''
        place_deal = ''
        buyer = ''
        city_buyer = ''
        dealer = ''
        city_dealer = ''
        source = ''
        items = item.split(',')
        #it is an art market, start of description
        art_market = items[0]
        art_market = art_market.strip()
        #is art market the only info?
        if len(items) ==1:
            proper_format = art_market
            readable_sent = f"Acquired on the {art_market}. "
            return proper_format, readable_sent
        #year of dealing
        year_test = items[1].replace('.', '').replace(' ', '').replace('-','')
        if year_test.isalnum():
            year = items[1]
            year = year.strip()
        #place of dealing
        if items[1].isalpha():
            place_deal = item[1]
            place_deal = place_deal.strip()
        #buyer or dealer (dealer working for buyer)
        for item in items:
            if 'to' in item:
                buyer = item
                buyer = buyer.strip()
                buyer_index = items.index(item)
                #city of buyer
                city_buyer = items[buyer_index+1]
                city_buyer = city_buyer.strip()
            if 'by' and 'dealer' in item:
                dealer = item
                dealer = dealer.strip()
                dealer_index = items.index(item)
                #city of dealer
                city_dealer = items[dealer_index+1]
                city_dealer = city_dealer.strip()
        items = item.split('{')
        for item in items:
            if '{' in item:
                source = item.replace('{', '').replace('}', '').strip()
                source = "{" +source+"}"
        #form proper formatted provenance sentence
        proper_format = art_market
        if year != '':
            proper_format = f"{proper_format}, {year}"
        if place_deal != '':
            proper_format = f"{proper_format}, {place_deal}"
        if buyer != '':
            proper_format = f"{proper_format}, {buyer}"
        if city_buyer != '':
            proper_format = f"{proper_format}, {city_buyer}"
        if dealer != '':
            proper_format = f"{proper_format}, {dealer}"
        if city_dealer != '':
            proper_format = f"{proper_format}, {city_dealer}"
        if source != '':
            proper_format = f"{proper_format} {source}"
        #print('Proper format:', proper_format)
        #form readable provenance sentence
        readable_sent = ''
        if 'acquired on' in art_market:
            art_market = art_market.replace('acquired on the', '')
        if 'from the' in art_market:
            art_market = art_market.replace('from the', '')
        #Acquired on the art market in year, place, by dealer, place, sold to, place
        if art_market != '':
            readable_sent = f"Acquired on the {art_market}"
        if year != '':
            readable_sent = f"{readable_sent} in {year}"
        if place_deal != '':
            readable_sent = f"{readable_sent}, {place_deal}"
        if dealer != '':
            readable_sent = f"{readable_sent}, {dealer}"
        if city_dealer != '':
            readable_sent = f"{readable_sent}, {city_dealer}"
        if buyer != '':
            readable_sent = f"{readable_sent}, sold {buyer}"
        if city_buyer != '':
            readable_sent = f"{readable_sent}, {city_buyer}"
        readable_sent = f"{readable_sent}. "
       # print('Readable sentence:',readable_sent)
    #art market dealer known
    else:
        dealer_name = ''
        place = ''
        date = ''
        date2 = ''
        source = '' #add source to previous part as well, just in case -> Still need to do this!!
        dealer_cat_source = ''
        #source dealer catalogue
        itemsA = item.split('}')
        for item in itemsA:
            if 'cat.' in item:
                dealer_cat_source = item.strip()
        #name(s) of dealer(s)
        items = item.split(',')
        if items[0] == ' dealer':
            dealer_name = items[1].strip()
        elif items[0] == 'dealer':
            dealer_name = items[1].strip()
        elif items[0] == 'Dealer':
            dealer_name = items[1].strip()
        elif items[0] == ' Dealer':
            dealer_name = items[1].strip()
        else:
            for item in items:
                if (item.startswith('dealers') or item.startswith('Dealers') or item.startswith(' dealers') or item.startswith(' Dealers')) and not 'cat.' in item:
                    if len(item) == (7 or 8):
                        dealer_name = item[1]
                    if len(item) > 9:
                        dealer_name = item.replace('dealers', '').replace('Dealers', '').replace(',', '').strip()
                elif (item.startswith('dealer') or item.startswith('Dealer') or item.startswith(' dealer') or item.startswith(' Dealer')) and not 'cat.' in item:
                    if len(item) == (6 or 7):
                        dealer_name = item[1]
                    if len(item) > 8:
                        dealer_name = item.replace('dealer', '').replace('Dealer','').replace(',','').strip()
                elif (item.startswith('dealers') or item.startswith('Dealers') or item.startswith(' dealers') or item.startswith(' Dealers')) and 'cat.' in item:
                    dealer_item = item[1]
                    dealer_item = dealer_item.split('}')
                    dealer_name = dealer_item[1]
                    dealer_name = dealer_item.replace('dealers', '').replace('Dealers', '').replace(',','').strip()
                elif (item.startswith('dealer') or item.startswith('Dealer') or item.startswith(' dealer') or item.startswith(' Dealer')) and 'cat.' in item:
                    dealer_item = item[1]
                    dealer_item = dealer_item.split('}')
                    dealer_name = dealer_item[1]
                    dealer_name = dealer_item.replace('dealer', '').replace('Dealer', '').replace(',','').strip()     
        #year and place
        if items[0] == ' dealer' and len(items) > 2:
            date_test = items[2].replace('.', '').replace(' ', '').replace('-','')
            if date_test.isalnum() and not date_test.isalpha():
                date = items[2]
                date = date.strip()
            if date_test.isalpha():
                place = items[2].strip()
            if place != '' and len(items) >3:
                date = items[3].strip()
            if len(items) > 4:
                if date != '':
                    date_test2 = items[3].replace('.', '').replace(' ', '').replace('-','')
                    if date_test2.isalnum() and not date_test2.isalpha():
                        date2 = items[4]
                        date2 = date2.strip()
        else:
            date_test = items[1].replace('.', '').replace(' ', '').replace('-','')
            if date_test.isalnum() and not date_test.isalpha():
                date = items[1]
                date = date.strip()
            if date_test.isalpha():
                place = items[1].strip()
            if place != '' and len(items) >2:
                date = items[2].strip()
            if len(items) > 3:
                if date != '':
                    date_test2 = items[2].replace('.', '').replace(' ', '').replace('-','')
                    if date_test2.isalnum() and not date_test2.isalpha():
                        date2 = items[3]
                        date2 = date2.strip()
        #misc source
        items = item.split('{')
        for item in items:
            if '}' in item and 'cat' not in item:
                source = item.replace('}', '').strip()
                source = "{" +source+"}"
        #form proper formatted sentence
        #structure is '{catalogue source} dealer(s), dealer_name, place, date, date2, source
        proper_format = ''
        if dealer_cat_source != '':
            proper_format = "{" + dealer_cat_source +"} "
        if 'and' in dealer_name:
            proper_format = f"{proper_format}dealers"
        if 'and' not in dealer_name:
            proper_format = f"{proper_format}dealer"
        if dealer_name != '':
            proper_format = f"{proper_format}, {dealer_name}"
        if place != '':
            proper_format = f"{proper_format}, {place}"
        if date != '':
            proper_format = f"{proper_format}, {date}"
        if date2 != '':
            proper_format = f"{proper_format}, {date2}"
        if source != '':
            proper_format = f"{proper_format} {source}"
        #form proper readable provenance sentence
        #Structure: acquired by the dealer(s) dealer_name, place, date, date2
        readable_sent = 'Acquired by the dealer(s)'
        if dealer_name != '':
            readable_sent = f"{readable_sent} {dealer_name}"
        if place != '':
            readable_sent = f"{readable_sent} in {place}"
        if date != '':
            readable_sent = f"{readable_sent}, {date}"
        if date2 != '':
            readable_sent =f"{readable_sent}, {date2}"
        readable_sent = f"{readable_sent}."
        
    return proper_format, readable_sent

In [55]:
#function for part of the inheritance items
def inheritance_other_item(item):
    """
    Parse provenance items inherited and return the new proper format, the readable sentence, and the format if the item is suspicious
    """
    
    test_item = item
    source = ''
    family_relation = ''
    name = ''
    name_extra = ''
    date = ''
    place = ''
    extra = ''
    to_name_item =''
    place_item = ''
    source_start = ''
    #certain or not?
    if '?' in item:
        proper_format = '?'
        readable_sent = 'Possibly inherited by'
    else:
        proper_format = ''
        readable_sent = 'Inherited by'
    #source
    if '}' in item or '{' in item:
        if item.startswith('{') or item.startswith(' {'):
            curly_split = item.split('}')
            source = curly_split[0].replace('{', '').strip()
            source = '{'+source+'}'
            source_start = source.strip()
        else:
            curly_split = item.split('{')
            if len(curly_split) >1:
                source = curly_split[1].replace('}','').strip()
                source = '{'+source+'}'
            curly_split2 = item.split('}')
            if ' his' in curly_split2[1] or ' her' in curly_split2[1]:
                source = curly_split2[0].strip()
    item = item.replace(source, '')
    #family relation
    comma_split = item.split(',')
    #if it starts with a source
    if '}' in comma_split[0]:
        source_split = comma_split[0].split('}')
        family_relation = source_split[1].replace('?', '').strip()
    else:
        family_relation = comma_split[0].replace('?', '').strip()
    #name and extra info on name
    name = comma_split[1].strip()
    if '(' in name:
        name_split = name.split('(')
        name = name_split[0].strip()
    if '(' not in name:
        name_extra = comma_split[2]
        name_extra = name_extra.split('(')
        name_extra = name_extra[0].strip()
    #date
    para_split = test_item.split('(')
    for items in para_split:
        if ')' in items:
            item = items.split(')')
            if '1' in item[0] or '2' in item[0]:
                date = item[0].strip()
    #place
    place = comma_split[-1].strip()
    if date in place:
        place = ''
    if '{' in place:
        place = place.split('{')
        place = place[0].strip()
    if name_extra in place:
        name_extra = ''
    #see if there is extra information missed
    test = test_item.replace(source,'')
    if ',' in test and place != '' and name !='':
        items = test.split(',')
        if len(items) > 1:
            for item in items:
                if name in item:
                    to_name_item = item
                if place in item:
                    place_item = item
            if (items.index(to_name_item)-items.index(place_item)) != -1:
                index = (items.index(to_name_item) +1)
                extra = items[index]
                extra = extra.strip()
                if items.index(to_name_item) +2 != items.index(place_item):
                    index2 = (items.index(to_name_item) +2)
                    extra = extra+', '+items[index2]
                    extra = extra.strip()
    if date in extra or to_name in extra:
        extra = ''
    #forming the sentences
    #form proper formatted provenance sentence
    if source_start != '':
        proper_format = f"{source_start}{proper_format}"
    if family_relation != '':
        proper_format = f"{proper_format} {family_relation},"
    if name != '':
        proper_format = f"{proper_format} {name}"
    if name_extra != '':
        proper_format = f"{proper_format}, {name_extra}"
    if date != '':
        proper_format = f"{proper_format} ({date})"
    if extra != '':
        proper_format = f"{proper_format}, {extra}"
    if place != '':
        proper_format = f"{proper_format}, {place}"
    if source != '' and source_start == '':
        proper_format = f"{proper_format} {source}"

    #form readable provenance sentence
    if family_relation != '':
        readable_sent = f"{readable_sent} {family_relation},"
    if name != '':
        readable_sent = f"{readable_sent} {name}"
    if name_extra != '':
        readable_sent = f"{readable_sent}, {name_extra}"
    if date != '':
        readable_sent = f"{readable_sent} ({date})"
    if extra != '':
        readable_sent = f"{readable_sent}, {extra}"
    if place != '':
        readable_sent = f"{readable_sent}, {place}"
    readable_sent = f"{readable_sent}."
    
    return proper_format, readable_sent

In [56]:
#function for other part of the inheritance items
def inheritance_descent_item(item):
    """
    Parse provenance items inherited (by descent) and return the new proper format, the readable sentence, and the format if the item is suspicious
    """
    
    test_item = item
    if '?' in item and not '?-' in item and not '-?' in item:
        proper_format = '? by descent'
        readable_sent = 'Possibly inherited by descent'
    else:
        proper_format = 'by descent'
        readable_sent = 'Inherited by descent'
    through = False
    through_name = ''
    family_relation = ''
    to_name = ''
    to_extra = ''
    date = ''
    place = ''
    source = ''
    extra = ''
    to_name_item =''
    place_item = ''
    if item.startswith('{') or item.startswith(' {'):
        curly_split = item.split('}')
        source = curly_split[0].replace('{', '').strip()
        source = '{'+source+'}'
    else:
        curly_split = item.split('{')
        if len(curly_split) >1:
            source = curly_split[1].replace('}','')
            source = '{'+source+'}'
    through_split = item.split(' through ')
    if len(through_split) != 1:
        through = True
        to_split = through_split[1].split(' to ')
        through_name = to_split[0].strip()
    test_split = test_item.replace(source,'')
    to_split = test_split.split(' to ')
    if len(to_split) >1:
        comma_split = to_split[1].split(',')
        if '1' not in comma_split[-1] or '2' not in comma_split[-1]:
            place = comma_split[-1].strip()
        for elem in comma_split:
            if len(comma_split) >1:
                if 'his ' in elem or 'her ' in elem:
                    family_relation = elem.strip()
                if family_relation != '':
                    to_name = comma_split[1].split('(')
                    to_name = to_name[0].strip()
                    if len(comma_split) >2:
                        if '(' in comma_split[2]:
                            to_extra = comma_split[2].split('(')
                            to_extra = to_extra[0].strip()
                if family_relation == '': 
                    to_name = comma_split[0].split('(')
                    to_name = to_name[0].strip()
                    if '(' in comma_split[1]:
                        to_extra = comma_split[1].split('(')
                        to_extra = to_extra[0].strip()
            if len(comma_split) == 1:
                to_split = comma_split[0].split(' to ')
                to_name = to_split[0].split('(')
                to_name = to_name[0].strip()
    para_split = item.split('(')
    for item in para_split:
        if ')' in item:
            comma_split = item.split(',')
            for item in comma_split:
                if item.replace('-', '').replace(')','').isnumeric():
                    date = item.replace(')', '').strip()
    if to_name in place:
        place = ''
    #see if there is extra information missed
    test = test_item.replace(source,'')
    if ',' in test and place != '' and to_name !='':
        items = test.split(',')
        if len(items) > 1:
            for item in items:
                if to_name in item:
                    to_name_item = item
                if place in item:
                    place_item = item
            if (items.index(to_name_item)-items.index(place_item)) != -1:
                index = (items.index(to_name_item) +1)
                extra = items[index]
                extra = extra.strip()
                if items.index(to_name_item) +2 != items.index(place_item):
                    index2 = (items.index(to_name_item) +2)
                    extra = extra+','+items[index2]
                    extra = extra.strip()
                    if items.index(to_name_item) +3 != items.index(place_item):
                        index3 = (items.index(to_name_item) +3)
                        extra = extra+','+items[index3]
                        extra = extra.strip()
    if date in extra or to_name in extra:
        extra = ''
    if date in to_extra:
        to_extra = to_extra.replace(date,'')
    #form proper formatted provenance sentence
    if test_item.startswith(source):
        proper_format = f"{source} {proper_format}"
    if through:
        proper_format = f"{proper_format} through {through_name}"
    if family_relation != '':
        proper_format = f"{proper_format} to {family_relation},"
    if to_name != '' and family_relation == '':
        proper_format = f"{proper_format} to {to_name}"
    if to_name != '' and family_relation != '':
        proper_format = f"{proper_format} {to_name}"
    if to_extra != '':
        proper_format = f"{proper_format}, {to_extra}"
    if date != '':
        proper_format = f"{proper_format} ({date})"
    if extra != '':
        proper_format = f"{proper_format}, {extra}"
    if place != '':
        proper_format = f"{proper_format}, {place}"
    if source != '' and not test_item.startswith(source):
        proper_format = f"{proper_format} {source}"
    #form readable provenance sentence
    if through:
        readable_sent = f"{readable_sent} through {through_name}"
    if family_relation != '':
        readable_sent = f"{readable_sent} by {family_relation},"
    if to_name != '' and family_relation == '':
        readable_sent =f"{readable_sent} by {to_name}"
    if to_name != '' and family_relation != '':
        readable_sent = f"{readable_sent}, {to_name}"
    if to_extra != '':
        readable_sent = f"{readable_sent}, {to_extra}"
    if date != '':
        readable_sent = f"{readable_sent} ({date})"
    if extra != '':
        readable_sent = f"{readable_sent}, {extra}"
    if place != '':
        readable_sent = f"{readable_sent}, {place}"
    readable_sent = f"{readable_sent}."
    
    return proper_format, readable_sent